[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EricBaidoo/GhanaSegNet/blob/main/notebooks/ghanasegnet_enhanced_colab.ipynb)

# GhanaSegNet (Enhanced) — Colab Run Notebook

This notebook is tailored to run ONLY the enhanced `GhanaSegNet` model on Google Colab.
It mounts Google Drive, copies or clones the repository, installs dependencies, checks GPU, runs the model readiness test, runs a short training sanity test (15 epochs), and saves outputs/checkpoints back to Drive.

**Before running:**
- Make sure your dataset is available in Google Drive and note the path (e.g. `/content/drive/MyDrive/dataset`).
- Optionally set `REPO_URL` below if you want to clone this repository from GitHub. Otherwise copy the repo into Drive and set `DRIVE_REPO_PATH`.
- Select *Runtime > Change runtime type* and pick GPU (preferably T4/P100/V100) for training.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Drive mounted at /content/drive')

In [ ]:
# Repo setup: either clone from GitHub or copy from Drive
# Edit REPO_URL or DRIVE_REPO_PATH below to point to your repo location in Drive
REPO_URL = ''  #@param {type: 'string'}
DRIVE_REPO_PATH = '/content/drive/MyDrive/GhanaSegNet'  #@param {type: 'string'}
import os, shutil, sys
TARGET = '/content/GhanaSegNet'
if REPO_URL:
    print('Cloning from', REPO_URL)
    !git clone {REPO_URL} {TARGET} || true
else:
    if os.path.exists(DRIVE_REPO_PATH):
        if os.path.exists(TARGET):
            shutil.rmtree(TARGET)
        shutil.copytree(DRIVE_REPO_PATH, TARGET)
        print('Copied repository from Drive:', DRIVE_REPO_PATH)
    else:
        print('ERROR: DRIVE_REPO_PATH not found. Either upload the repo to Drive or set REPO_URL to clone.')

# Add repo to PYTHONPATH
sys.path.append(TARGET)
print('Repository is available at', TARGET)

In [ ]:
# Install dependencies from repository requirements.txt (may take a few minutes)
import os
req = '/content/GhanaSegNet/requirements.txt'
if os.path.exists(req):
    print('Installing from', req)
    !pip install -r {req}
else:
    print('requirements.txt not found in repo; you can pip install dependencies manually')

# Print torch info (Colab usually has a compatible torch preinstalled)
import torch
print('PyTorch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('Device name:', torch.cuda.get_device_name(torch.cuda.current_device()))

In [ ]:
# Quick environment checks and PYTHONPATH setup
import sys, os
ROOT = '/content/GhanaSegNet'
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)
os.environ['PYTHONPATH'] = ROOT + ':' + os.environ.get('PYTHONPATH', '')
print('PYTHONPATH updated:', os.environ['PYTHONPATH'].split(':')[0])

# Confirm we can import the enhanced model and test scripts
try:
    from models.ghanasegnet import GhanaSegNet
    print('✅ Imported GhanaSegNet')
except Exception as e:
    print('❌ Import error:', e)

In [ ]:
# Run the pre-made readiness test (creates model, runs forward pass and prints parameter count)
print('Running readiness test...')
!python /content/GhanaSegNet/test_enhanced_architecture.py

In [ ]:
# QUICK TRAIN: run a short 15-epoch sanity training run for the enhanced model
# Edit DATA_DIR to point to your dataset in Drive (ensure GhanaFoodDataset loader uses this path)
DATA_DIR = '/content/drive/MyDrive/dataset'  #@param {type: 'string'}
print('Using dataset path:', DATA_DIR)

# Change working directory to the repo root so imports resolve correctly
import os, sys
repo_dir = '/content/GhanaSegNet'
if os.path.exists(repo_dir):
    os.chdir(repo_dir)
    print('Changed CWD to', repo_dir)
else:
    print('ERROR: repo directory not found at', repo_dir)
    raise SystemExit(1)

# Direct function call - much simpler than argument parsing!
from scripts.train_baselines import train_model

# Training configuration
config = {
    'epochs': 15,
    'batch_size': 8,
    'learning_rate': 1e-4,
    'weight_decay': 1e-4,
    'num_classes': 6,
    'custom_seed': None,
    'benchmark_mode': True,
    'dataset_path': DATA_DIR,
    'device': 'cuda',
    'timestamp': '2025-10-12',
    'note': 'Enhanced GhanaSegNet for 30% mIoU target'
}

print('Starting enhanced GhanaSegNet training...')
print(f'Config: {config}')

try:
    result = train_model('ghanasegnet', config)
    print(f'🎯 TRAINING COMPLETED! Best IoU: {result["best_iou"]:.4f} ({result["best_iou"]*100:.2f}%)')
    if result['best_iou'] >= 0.30:
        print('🏆 30% mIoU TARGET ACHIEVED!')
    else:
        print(f'📈 Progress: {result["best_iou"]*100:.2f}% (Target: 30%)')
except Exception as e:
    print(f'❌ Training failed: {e}')
    import traceback
    traceback.print_exc()

## After the quick test
- If the quick 15-epoch run succeeds and logs look good, run the full benchmark by changing `epochs: 80` in the config above.
- To save checkpoints and logs to Drive, use the copy-outputs cell below.

Example full-run config (change epochs in the cell above):
```python
config = {
    'epochs': 80,  # <-- Change this for full training
    'batch_size': 8,
    'learning_rate': 1e-4,
    # ... rest stays the same
}
```

In [ ]:
# OPTIONAL: copy outputs to Drive after run (adjust paths if your script writes elsewhere)
OUT_DIR = '/content/drive/MyDrive/GhanaSegNet_outputs'
import os, shutil
os.makedirs(OUT_DIR, exist_ok=True)
print('Copying outputs to', OUT_DIR)
if os.path.exists('/content/GhanaSegNet/outputs'):
    for item in os.listdir('/content/GhanaSegNet/outputs'):
        src = os.path.join('/content/GhanaSegNet/outputs', item)
        dst = os.path.join(OUT_DIR, item)
        if os.path.isdir(src):
            shutil.copytree(src, dst, dirs_exist_ok=True)
        else:
            shutil.copy2(src, dst)
    print('Outputs copied.')
else:
    print('No outputs directory found at /content/GhanaSegNet/outputs')

### Notes & Troubleshooting
- If `pip install -r requirements.txt` fails due to CUDA mismatch, install a colab-friendly torch wheel first (e.g., `pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118`).
- If your dataset loader expects a different folder layout, either modify `DATA_DIR` or edit `scripts/train_baselines.py` to point to the correct path.
- Use *Runtime > Restart runtime and run all* if you change installed packages.